# Install and Run the CD-HIT

In [ ]:
!sudo apt-get update
!sudo apt-get install build-essential wget zlib1g-dev
!wget https://github.com/weizhongli/cdhit/archive/refs/tags/V4.8.1.tar.gz -O cdhit.tar.gz
!tar -xvf cdhit.tar.gz
!cd cdhit-4.8.1 && make
!mkdir -p /content/cdhit_bin
!cp cdhit-4.8.1/cd-hit* /content/cdhit_bin/
!cp -r cdhit-4.8.1/cd-hit-auxtools /content/cdhit_bin/

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,188 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,527 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

In [ ]:
pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 43.1 MB/s eta 0:00:00


In [ ]:
!/content/cdhit_bin/cd-hit -h

		====== CD-HIT version 4.8.1 (built on Dec  4 2024) ======

Usage: /content/cdhit_bin/cd-hit [Options] 

Options

   -i	input filename in fasta format, required, can be in .gz format
   -o	output filename, required
   -c	sequence identity threshold, default 0.9
 	this is the default cd-hit's "global sequence identity" calculated as:
 	number of identical amino acids or bases in alignment
 	divided by the full length of the shorter sequence
   -G	use global sequence identity, default 1
 	if set to 0, then use local sequence identity, calculated as :
 	number of identical amino acids or bases in alignment
 	divided by the length of the alignment
 	NOTE!!! don't use -G 0 unless you use alignment coverage controls
 	see options -aL, -AL, -aS, -AS
   -b	band_width of alignment, default 20
   -M	memory limit (in MB) for the program, default 800; 0 for unlimitted;
   -T	number of threads, default 1; with 0, all CPUs will be used
   -n	word_length, default 5, see user's guide for choosing it


In [ ]:
!/content/cdhit_bin/cd-hit -i ZN_samples.fasta -o clustered_ZN.fasta -c 0.4 -n 2 # the input file name should be changed based on Pre_processing_1

Program: CD-HIT, V4.8.1 (+OpenMP), Dec 04 2024, 00:49:58
Command: /content/cdhit_bin/cd-hit -i ZN_samples.fasta -o
         clustered_ZN.fasta -c 0.4 -n 2

Started: Wed Dec  4 00:50:42 2024
                            Output                              
----------------------------------------------------------------
total seq: 481
longest and shortest : 667 and 55
Total letters: 130070
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 1 X 10M = 10M
Table           : 1 X 0M = 0M
Miscellaneous   : 0M
Total           : 10M

Table limit with the given memory limit:
Max number of representatives: 1743342
Max number of word counting entries: 98644199

comparing sequences from          0  to        481

      481  finished        118  clusters

Approximated maximum memory consumption: 10M
writing new database
writing clustering information
program completed !

Total CPU time 0.52


# Splitting the data into training and testing sets

In [ ]:
import random
from Bio import SeqIO

# Read cluster file and create a dictionary mapping sequences to clusters
def parse_cluster_file(cluster_file):
    clusters = {}
    current_cluster = None
    with open(cluster_file, 'r') as f:
        for line in f:
            if line.startswith('>Cluster'):
                current_cluster = line.strip()
                clusters[current_cluster] = []
            else:
                sequence_id = line.strip().split(">")[1].split("...")[0]
                clusters[current_cluster].append(sequence_id)
    return clusters

# Split clusters into training and testing sets
def split_clusters(clusters, train_ratio=0.8):
    cluster_keys = list(clusters.keys())
    random.shuffle(cluster_keys)
    split_point = int(train_ratio * len(cluster_keys))
    train_clusters = {key: clusters[key] for key in cluster_keys[:split_point]}
    test_clusters = {key: clusters[key] for key in cluster_keys[split_point:]}
    return train_clusters, test_clusters

# Write sequences to fasta file based on clusters
def write_fasta_file(original_fasta_file, clusters, output_file):
    sequences = SeqIO.to_dict(SeqIO.parse(original_fasta_file, "fasta"))
    selected_sequences = []

    for cluster in clusters.values():
        for seq_id in cluster:
            if seq_id in sequences:
                selected_sequences.append(sequences[seq_id])

    SeqIO.write(selected_sequences, output_file, "fasta")

# Main script
cluster_file = 'clustered_ZN.fasta.clstr'  # should be changed based on the above
original_fasta_file = 'clustered_ZN.fasta' # should be changed based on the above
all_sequences_file = 'ZN_samples.fasta'  # the input file name should be changed based on Pre_processing_1 (all original sequences file without hashes)
train_output_file = 'train_set.fasta'
test_output_file = 'test_set.fasta'

# Parse the cluster file
clusters = parse_cluster_file(cluster_file)

# Split clusters into training and testing sets
train_clusters, test_clusters = split_clusters(clusters)

# Write training and testing fasta files
write_fasta_file(all_sequences_file, train_clusters, train_output_file)
write_fasta_file(all_sequences_file, test_clusters, test_output_file)

print(f'Training set saved to {train_output_file}')
print(f'Testing set saved to {test_output_file}')


Training set saved to train_set.fasta
Testing set saved to test_set.fasta


#Splitting the data into training, evaluating, and testing

In [ ]:
import random
from Bio import SeqIO

# Read cluster file and create a dictionary mapping sequences to clusters
def parse_cluster_file(cluster_file):
    clusters = {}
    current_cluster = None
    with open(cluster_file, 'r') as f:
        for line in f:
            if line.startswith('>Cluster'):
                current_cluster = line.strip()
                clusters[current_cluster] = []
            else:
                sequence_id = line.strip().split(">")[1].split("...")[0]
                clusters[current_cluster].append(sequence_id)
    return clusters

# Split clusters into training, evaluation, and testing sets
def split_clusters(clusters, train_ratio=0.7, eval_ratio=0.2):
    cluster_keys = list(clusters.keys())
    random.shuffle(cluster_keys)
    train_split_point = int(train_ratio * len(cluster_keys))
    eval_split_point = int((train_ratio + eval_ratio) * len(cluster_keys))

    train_clusters = {key: clusters[key] for key in cluster_keys[:train_split_point]}
    eval_clusters = {key: clusters[key] for key in cluster_keys[train_split_point:eval_split_point]}
    test_clusters = {key: clusters[key] for key in cluster_keys[eval_split_point:]}

    return train_clusters, eval_clusters, test_clusters

# Write sequences to fasta file based on clusters
def write_fasta_file(original_fasta_file, clusters, output_file):
    sequences = SeqIO.to_dict(SeqIO.parse(original_fasta_file, "fasta"))
    selected_sequences = []

    for cluster in clusters.values():
        for seq_id in cluster:
            if seq_id in sequences:
                selected_sequences.append(sequences[seq_id])

    SeqIO.write(selected_sequences, output_file, "fasta")

# Main script
cluster_file = 'clustered_ZN.fasta.clstr'
original_fasta_file = 'ZN_samples.fasta'
train_output_file = 'train_set.fasta'
eval_output_file = 'eval_set.fasta'
test_output_file = 'test_set.fasta'

# Parse the cluster file
clusters = parse_cluster_file(cluster_file)

# Split clusters into training, evaluation, and testing sets
train_clusters, eval_clusters, test_clusters = split_clusters(clusters)

# Write training, evaluation, and testing fasta files
write_fasta_file(original_fasta_file, train_clusters, train_output_file)
write_fasta_file(original_fasta_file, eval_clusters, eval_output_file)
write_fasta_file(original_fasta_file, test_clusters, test_output_file)

print(f'Training set saved to {train_output_file}')
print(f'Evaluation set saved to {eval_output_file}')
print(f'Testing set saved to {test_output_file}')


Training set saved to train_set.fasta
Evaluation set saved to eval_set.fasta
Testing set saved to test_set.fasta


#Splitting the labels (training and testing)

In [ ]:
from Bio import SeqIO

# Function to extract labels
def extract_labels(fasta_file, labels_file, output_file):
    # Read sequence IDs from the fasta file
    sequence_ids = [record.id for record in SeqIO.parse(fasta_file, "fasta")]

    # Open labels file and extract the labels for the corresponding sequence IDs
    with open(labels_file, 'r') as labels_f, open(output_file, 'w') as out_f:
        write_label = False
        for line in labels_f:
            if line.startswith('>'):
                seq_id = line.strip().split('>')[1]
                if seq_id in sequence_ids:
                    write_label = True
                    out_f.write(line)
                else:
                    write_label = False
            elif write_label:
                out_f.write(line)

# File paths
train_fasta = 'train_set.fasta'
test_fasta = 'test_set.fasta'
labels_fasta = 'ZN_labels.fasta'  # this line should be changed

# Output files
train_labels_output = 'train_labels.fasta'
test_labels_output = 'test_labels.fasta'

# Extract labels
extract_labels(train_fasta, labels_fasta, train_labels_output)
extract_labels(test_fasta, labels_fasta, test_labels_output)

print("Labels extracted and saved to respective files.")

Labels extracted and saved to respective files.


#Splitting the labels (training, evaluating, and testing)

In [ ]:
from Bio import SeqIO

# Function to extract labels
def extract_labels(fasta_file, labels_file, output_file):
    # Read sequence IDs from the fasta file
    sequence_ids = [record.id for record in SeqIO.parse(fasta_file, "fasta")]

    # Open labels file and extract the labels for the corresponding sequence IDs
    with open(labels_file, 'r') as labels_f, open(output_file, 'w') as out_f:
        write_label = False
        for line in labels_f:
            if line.startswith('>'):
                seq_id = line.strip().split('>')[1]
                if seq_id in sequence_ids:
                    write_label = True
                    out_f.write(line)
                else:
                    write_label = False
            elif write_label:
                out_f.write(line)

# File paths
train_fasta = 'train_set.fasta'
eval_fasta = 'eval_set.fasta'
test_fasta = 'test_set.fasta'
labels_fasta = 'ZN_labels.fasta'  # Update this line if needed

# Output files
train_labels_output = 'train_labels.fasta'
eval_labels_output = 'eval_labels.fasta'
test_labels_output = 'test_labels.fasta'

# Extract labels for each set
extract_labels(train_fasta, labels_fasta, train_labels_output)
extract_labels(eval_fasta, labels_fasta, eval_labels_output)
extract_labels(test_fasta, labels_fasta, test_labels_output)

print("Labels extracted and saved to respective files.")


Labels extracted and saved to respective files.


#Sorting (training and testing)

In [ ]:
from Bio import SeqIO

def sort_fasta_sequences(input_file, output_file):
    # Read the sequences from the input file
    sequences = list(SeqIO.parse(input_file, "fasta"))

    # Sort the sequences by the sequence id
    sequences.sort(key=lambda x: x.id)

    # Write the sorted sequences to the output file
    SeqIO.write(sequences, output_file, "fasta")

# Define the input and output files for the training set
train_input_file = "train_set.fasta"
train_output_file = "sorted_train_set.fasta"
train_labels_input_file = "train_labels.fasta"
train_labels_output_file = "sorted_train_labels.fasta"

# Define the input and output files for the test set
test_input_file = "test_set.fasta"
test_output_file = "sorted_test_set.fasta"
test_labels_input_file = "test_labels.fasta"
test_labels_output_file = "sorted_test_labels.fasta"

# Sort the sequences in the training set
sort_fasta_sequences(train_input_file, train_output_file)
sort_fasta_sequences(train_labels_input_file, train_labels_output_file)

# Sort the sequences in the test set
sort_fasta_sequences(test_input_file, test_output_file)
sort_fasta_sequences(test_labels_input_file, test_labels_output_file)

print("Sorting completed and files saved.")


Sorting completed and files saved.


#Sorting (training, evaluating, and testing)

In [ ]:
from Bio import SeqIO

def sort_fasta_sequences(input_file, output_file):
    # Read the sequences from the input file
    sequences = list(SeqIO.parse(input_file, "fasta"))

    # Sort the sequences by the sequence id
    sequences.sort(key=lambda x: x.id)

    # Write the sorted sequences to the output file
    SeqIO.write(sequences, output_file, "fasta")

# Define the input and output files for the training set
train_input_file = "train_set.fasta"
train_output_file = "sorted_train_set.fasta"
train_labels_input_file = "train_labels.fasta"
train_labels_output_file = "sorted_train_labels.fasta"

# Define the input and output files for the evaluation set
eval_input_file = "eval_set.fasta"
eval_output_file = "sorted_eval_set.fasta"
eval_labels_input_file = "eval_labels.fasta"
eval_labels_output_file = "sorted_eval_labels.fasta"

# Define the input and output files for the test set
test_input_file = "test_set.fasta"
test_output_file = "sorted_test_set.fasta"
test_labels_input_file = "test_labels.fasta"
test_labels_output_file = "sorted_test_labels.fasta"

# Sort the sequences in the training set
sort_fasta_sequences(train_input_file, train_output_file)
sort_fasta_sequences(train_labels_input_file, train_labels_output_file)

# Sort the sequences in the evaluation set
sort_fasta_sequences(eval_input_file, eval_output_file)
sort_fasta_sequences(eval_labels_input_file, eval_labels_output_file)

# Sort the sequences in the test set
sort_fasta_sequences(test_input_file, test_output_file)
sort_fasta_sequences(test_labels_input_file, test_labels_output_file)

print("Sorting completed and files saved.")


Sorting completed and files saved.


#Check that sequences from the same cluster are not split across the training, evaluation, and testing sets

In [ ]:
from Bio import SeqIO

def parse_cluster_file(cluster_file):
    clusters = {}
    current_cluster = None
    with open(cluster_file, 'r') as f:
        for line in f:
            if line.startswith('>Cluster'):
                current_cluster = line.strip()
                clusters[current_cluster] = []
            else:
                sequence_id = line.strip().split(">")[1].split("...")[0]
                clusters[current_cluster].append(sequence_id)
    return clusters

def check_cluster_integrity(train_file, eval_file, test_file, cluster_file):
    # Load sequence IDs from each set
    train_ids = {record.id for record in SeqIO.parse(train_file, "fasta")}
    eval_ids = {record.id for record in SeqIO.parse(eval_file, "fasta")}
    test_ids = {record.id for record in SeqIO.parse(test_file, "fasta")}

    # Print total number of sequences in each set
    print(f"Total sequences in Training set: {len(train_ids)}")
    print(f"Total sequences in Evaluation set: {len(eval_ids)}")
    print(f"Total sequences in Testing set: {len(test_ids)}")

    # Parse cluster information
    clusters = parse_cluster_file(cluster_file)

    # Check that no sequences from the same cluster are in different sets
    for cluster, seq_ids in clusters.items():
        sets = set()
        for seq_id in seq_ids:
            if seq_id in train_ids:
                sets.add('Training')
            if seq_id in eval_ids:
                sets.add('Evaluation')
            if seq_id in test_ids:
                sets.add('Testing')

        if len(sets) > 1:
            print(f"Cluster {cluster} spans multiple sets: {sets} (Number of sequences: {len(seq_ids)})")
        else:
            print(f"Cluster {cluster} is only in one set: {sets} (Number of sequences: {len(seq_ids)})")

# File paths
train_file = 'sorted_train_set.fasta'
eval_file = 'sorted_eval_set.fasta'
test_file = 'sorted_test_set.fasta'
cluster_file = 'clustered_ZN.fasta.clstr'

# Perform the check
check_cluster_integrity(train_file, eval_file, test_file, cluster_file)


Total sequences in Training set: 377
Total sequences in Evaluation set: 63
Total sequences in Testing set: 41
Cluster >Cluster 0 is only in one set: {'Training'} (Number of sequences: 2)
Cluster >Cluster 1 is only in one set: {'Training'} (Number of sequences: 2)
Cluster >Cluster 2 is only in one set: {'Testing'} (Number of sequences: 10)
Cluster >Cluster 3 is only in one set: {'Training'} (Number of sequences: 1)
Cluster >Cluster 4 is only in one set: {'Training'} (Number of sequences: 31)
Cluster >Cluster 5 is only in one set: {'Training'} (Number of sequences: 2)
Cluster >Cluster 6 is only in one set: {'Training'} (Number of sequences: 2)
Cluster >Cluster 7 is only in one set: {'Training'} (Number of sequences: 1)
Cluster >Cluster 8 is only in one set: {'Training'} (Number of sequences: 1)
Cluster >Cluster 9 is only in one set: {'Training'} (Number of sequences: 2)
Cluster >Cluster 10 is only in one set: {'Testing'} (Number of sequences: 2)
Cluster >Cluster 11 is only in one set: {'